In [1]:
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [39]:
train_df = pd.read_excel("data/train.xlsx")
test1_df = pd.read_excel("data/testA.xlsx")
test2_df = pd.read_excel("data/testB.xlsx")

In [40]:
train_y = train_df[train_df.columns[-1]]
train_y.shape

(800,)

In [41]:
train_df = train_df[train_y>1.8]
train_y = train_y[train_y>1.8]
print(train_df.shape, train_y.shape)

(799, 5954) (799,)


In [42]:
new_column = test1_df.columns
train_df = train_df[new_column]
train_id = train_df['ID']
test1_id = test1_df['ID']
test2_id = test2_df['ID']
train_df.drop(['ID'], axis=1, inplace=True)
test1_df.drop(['ID'], axis=1, inplace=True)
test2_df.drop(['ID'], axis=1, inplace=True)

In [45]:
names = train_df.columns
dts = train_df.dtypes
drop_names = []
year = []
ints = []
objs = []
ratio = 0.1
for i in range(len(names)):
    n = names[i]
    tp = dts[i]
    trd = train_df[n]
    ted = test1_df[n]
    if 'object' == str(tp):
        objs.append(n)
    else:
        misstr = np.sum(trd.isnull())/799
        misste = np.sum(ted.isnull())/300
        if np.abs(misstr-misste) > ratio:
            drop_names.append(n)
        else:
            trrd = trd[trd.notnull()].values
            terd = ted[ted.notnull()].values
            if len(trrd) == 0 or len(terd) == 0:
                drop_names.append(n)
                continue
            if (str(trrd[0])[:4] == '2017' and str(trrd[700])[:4] == '2017') or (str(trrd[0])[:4] == '2016' and str(trrd[700])[:4] == '2016'):
                    year.append(n)
            elif 'int' in str(tp):
                ints.append(n)
            else:
                m1 = np.mean(trrd)
                s1 = np.std(trrd)
                m2 = np.mean(terd)
                s2 = np.std(terd)
                if np.abs(m1-m2)>ratio*np.abs(m1) and np.abs(s1-s2)>ratio*np.abs(s1):
                    drop_names.append(n)
print(len(drop_names),len(year), len(ints), len(objs))
print()

1777 40 629 11



In [46]:
train_df.drop(drop_names, axis=1, inplace=True)
test1_df.drop(drop_names, axis=1, inplace=True)
test2_df.drop(drop_names, axis=1, inplace=True)

In [47]:
df_X = pd.concat([train_df, test1_df, test2_df], axis=0)

In [48]:
np.sum(df_X[objs].isnull().values)

0

In [49]:
ints += objs
print(len(ints))

640


In [50]:
remains1 = []
remains2 = []
tobedrop = []

for n in ints:
    lu1 = len(train_df[n].unique())
    lu2 = len(df_X[n].unique())
    if lu1 < 15 and lu1 > 1:
        remains1.append(n)
    if lu2 < 15 and lu2 > 1:
        remains2.append(n)
    if lu1 != lu2:
        tobedrop.append(n)
    
print(len(remains1), len(remains2), len(tobedrop), len(year))

177 244 277 40


In [51]:
toberemain = list(set(remains1) & set(remains2) - set(tobedrop))
print(len(toberemain))

86


In [52]:
def dict2list(dic:dict):
    ''' 将字典转化为列表 '''
    keys = dic.keys()
    vals = dic.values()
    lst = [(key, val) for key, val in zip(keys, vals)]
    return lst

def LastRemainJudge(column):
    unique = set(column)
    uf = {}
    for u in unique:
        uf[u] = 0
    for c in column:
        uf[c] += 1
    sl = sorted(dict2list(uf), key = lambda x:x[1], reverse=True)
    x = sl[0][1]/len(column)
    sl = np.array(sl)
    if x < 0.8:
        #print(sl[:, 1]/len(column))
        return True
    else:
        return False

In [53]:
lm1 = []
lm2 = []
for n in toberemain:
    if LastRemainJudge(train_df[n]):
        lm1.append(n)
    if LastRemainJudge(df_X[n]):
        lm2.append(n)
print(len(lm1), len(lm2), len(set(lm1)&set(lm2)))

54 53 53


In [54]:
names = df_X.columns
dts = df_X.dtypes
dfx_float_n = []
dfx_onehot_n = list(set(lm1) & set(lm2))
print(len(dfx_onehot_n))
for i in range(len(names)):
    n = names[i]
    d = dts[i]
    if 'float' in str(d):
        dfx_float_n.append(n)
print(len(dfx_float_n))
dfx_float = df_X[dfx_float_n]
dfx_onehot = df_X[dfx_onehot_n]
print(dfx_float.shape, dfx_onehot.shape)

53
3646
(1511, 3646) (1511, 53)


In [55]:
floatyear = []
for n in dfx_float.columns:
    c = dfx_float[n]
    r = c[c.notnull()].astype(np.int64).values
    if (str(r[0])[:4] == '2017' and str(r[-1])[:4] == '2017') or (str(r[0])[:4] == '2016' and str(r[-1])[:4] == '2016'):
        floatyear.append(n)
print(floatyear)

['210X24', '210X205', '210X213', '210X215', '220X71', '300X2', '300X3', '300X4', '300X6', '300X7', '300X9', '300X10', '300X13', '300X14', '300X20', '311X6', '311X7', '311X20', '311X22', '311X55', '311X56', '311X59', '311X60', '311X78', '311X79', '311X163', '311X164', '311X170', '311X171', '400X7', '400X9', '400X25', '400X27', '400X60', '400X61', '400X64', '400X65', '400X83', '400X84', '400X168', '400X169', '400X219', '400X220', '420X186', '520X148', '520X152', '520X171']


In [56]:
dfxt = dfx_float[floatyear]
for n in dfxt.columns:
    x = np.sum(dfxt[n].isnull())
    if x > 0:
        c = dfxt[n]
        x1 = np.sum(dfxt[n][:799].isnull())
        if x1 > 0:
            dfxt[:799][c[:799].isnull()] = np.mean(c[:799][c[:799].notnull()])
        x2 = np.sum(dfxt[n][799:1099].isnull())
        if x2 > 0:
            dfxt[799:1099][c[799:1099].isnull()] = np.mean(c[799:1099][c[799:1099].notnull()])
        x3 = np.sum(dfxt[n][1099:].isnull())
        if x3 > 0:
            dfxt[1099:][c[1099:].isnull()] = np.mean(c[1099:][c[1099:].notnull()])
        print(n, x, x1, x2, x3)

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/magnusterra/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2444: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer(indexer, value)
/home/magnusterra/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2424: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  

210X24 1 1 0 0


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


210X213 4 3 1 0
220X71 1 1 0 0


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


300X2 45 27 9 9
300X3 73 50 9 14
300X4 140 95 17 28
300X6 1 1 0 0
300X7 44 29 9 6
300X9 13 9 3 1
300X10 15 10 2 3
300X13 2 1 0 1
300X14 6 3 2 1
311X6 58 28 11 19
311X7 194 111 41 42
311X20 177 103 34 40
311X55 35 18 12 5
311X56 40 24 4 12
311X59 13 9 2 2
311X60 39 23 9 7
311X163 2 0 2 0
311X164 8 6 1 1
311X170 1 1 0 0
311X171 18 13 0 5
400X7 1 1 0 0
400X9 1 0 1 0
400X25 2 2 0 0
400X60 1 1 0 0
400X64 1 0 1 0
400X83 1 0 1 0
400X169 1 1 0 0
520X152 1 1 0 0


In [57]:
dfx_year = df_X[year]
print(np.sum(dfx_year.isnull().values))

1


In [58]:
for n in dfx_year.columns:
    c = dfx_year[n]
    x = np.sum(c.isnull())
    if x > 0:
        x1 = np.sum(c[:799].isnull())
        x2 = np.sum(c[799:1099].isnull())
        x3 = np.sum(c[1099:].isnull())
        c = dfx_year[n]
        dfx_year[1099:][c[1099:].isnull()] = np.mean(c[1099:][c[1099:].notnull()])
        print(n, x, x1, x2, x3)

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/magnusterra/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


520X148 1 0 0 1


In [59]:
print(np.sum(dfxt.isnull().values), np.sum(dfx_year.isnull().values))

0 0


In [60]:
dfx_float.drop(floatyear, axis=1, inplace=True)
print(dfx_float.shape)

(1511, 3599)


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [61]:
dfx_days = pd.concat([dfx_year, dfxt], axis=1)
print(dfx_days.shape)

(1511, 87)


In [62]:
c = copy.copy(dfx_days['520X148'])
print(c.shape)

(1511, 2)


In [63]:
dfx_days.drop(['520X148'], axis=1, inplace=True)
print(dfx_days.shape)
dfx_days['520X148'] = c.values[:, 0]
dfx_days['520X148X1'] = c.values[:, 1]

(1511, 85)


In [64]:
#convert type of ime columns into int so that the length is true
names = dfx_days.columns
dtypes = dfx_days.dtypes
for i in range(len(names)):
    n = names[i]
    if 'int' not in str(dtypes[i]):
        dfx_days[n] = dfx_days[n].astype(np.int64)
    print(i, n, dfx_days[n].dtype)

0 210X204 int64
1 220X67 int64
2 220X75 int64
3 220X79 int64
4 220X83 int64
5 220X87 int64
6 220X91 int64
7 220X95 int64
8 310X56 int64
9 310X60 int64
10 310X64 int64
11 310X68 int64
12 310X72 int64
13 310X76 int64
14 310X80 int64
15 310X84 int64
16 360X710 int64
17 360X711 int64
18 360X1287 int64
19 360X1291 int64
20 360X1292 int64
21 360X1293 int64
22 420X7 int64
23 420X9 int64
24 420X25 int64
25 420X27 int64
26 520X173 int64
27 520X248 int64
28 520X250 int64
29 520X346 int64
30 520X348 int64
31 520X354 int64
32 520X356 int64
33 750X710 int64
34 750X711 int64
35 750X1287 int64
36 750X1291 int64
37 750X1292 int64
38 750X1293 int64
39 210X24 int64
40 210X205 int64
41 210X213 int64
42 210X215 int64
43 220X71 int64
44 300X2 int64
45 300X3 int64
46 300X4 int64
47 300X6 int64
48 300X7 int64
49 300X9 int64
50 300X10 int64
51 300X13 int64
52 300X14 int64
53 300X20 int64
54 311X6 int64
55 311X7 int64
56 311X20 int64
57 311X22 int64
58 311X55 int64
59 311X56 int64
60 311X59 int64
61 311X60 int

In [65]:
copydfxday = copy.copy(dfx_days)

In [66]:
day_n = dfx_days.columns
standardcolumn = copy.copy(dfx_days[day_n[1]])
standardlen = len(str(standardcolumn[0]))
print(standardlen)
drop_names = []
for i in range(len(day_n)):
    #if i == 1:
    #    print(i, "standard")
    #    continue
    l = len(str(dfx_days[day_n[i]][0]))
    bias = standardlen - l
    b10 = 1
    if bias > 0:
        for j in range(bias):
            b10 *= 10
    elif bias < 0:
        for j in range(-bias):
            b10 /= 10
    print(i, l, bias, b10)
    if b10 != 1:
        dfx_days[day_n[i]] *= b10
    dfx_days[day_n[i]] = dfx_days[day_n[i]] - standardcolumn
    if np.sum(dfx_days[day_n[i]]) == 0:
        drop_names.append(day_n[i])
print(drop_names)

92
0 87 5 100000
1 92 0 1
2 92 0 1
3 92 0 1
4 92 0 1
5 92 0 1
6 92 0 1
7 92 0 1
8 92 0 1
9 92 0 1
10 92 0 1
11 92 0 1
12 92 0 1
13 92 0 1
14 92 0 1
15 92 0 1
16 69 23 100000000000000000000000
17 69 23 100000000000000000000000
18 70 22 10000000000000000000000
19 70 22 10000000000000000000000
20 70 22 10000000000000000000000
21 70 22 10000000000000000000000
22 85 7 10000000
23 85 7 10000000
24 86 6 1000000
25 86 6 1000000
26 87 5 100000
27 87 5 100000
28 87 5 100000
29 87 5 100000
30 87 5 100000
31 87 5 100000
32 87 5 100000
33 69 23 100000000000000000000000
34 69 23 100000000000000000000000
35 70 22 10000000000000000000000
36 70 22 10000000000000000000000
37 70 22 10000000000000000000000
38 70 22 10000000000000000000000
39 86 6 1000000
40 87 5 100000
41 87 5 100000
42 87 5 100000
43 92 0 1
44 85 7 10000000
45 85 7 10000000
46 85 7 10000000
47 85 7 10000000
48 85 7 10000000
49 85 7 10000000
50 86 6 1000000
51 86 6 1000000
52 86 6 1000000
53 86 6 1000000
54 85 7 10000000
55 85 7 10000000


In [67]:
dfx_days.drop(drop_names, axis=1, inplace=True)
print(dfx_days.shape)

(1511, 86)


In [68]:
# normalize time
names = dfx_days.columns
for n in names:
    c = dfx_days[n]
    tmpm = np.mean(c)
    tmps = np.std(c)
    dfx_days[n] = (dfx_days[n] - tmpm) / tmps

In [69]:
tooshort = []
for n in names:
    l = len(set(dfx_days[n]))
    if l < 100:
        print(n, l)
        tooshort.append(n)
print(tooshort)

360X710 47
360X711 47
360X1287 47
360X1291 47
360X1292 47
360X1293 47
750X710 45
750X711 45
750X1287 45
750X1291 45
750X1292 45
750X1293 45
['360X710', '360X711', '360X1287', '360X1291', '360X1292', '360X1293', '750X710', '750X711', '750X1287', '750X1291', '750X1292', '750X1293']


In [70]:
dfxoh = pd.concat([dfx_onehot, copydfxday[tooshort]], axis=1)
print(dfxoh.shape)

(1511, 65)


In [71]:
# make onehots onehot
ohnames = dfxoh.columns
for i in range(len(ohnames)):
    n = ohnames[i]
    tmp = pd.get_dummies(dfxoh[n], prefix='onehot_'+str(i))
    dfxoh = pd.concat([dfxoh, tmp], axis=1)
    print(dfxoh.shape)

(1511, 74)
(1511, 76)
(1511, 79)
(1511, 85)
(1511, 87)
(1511, 90)
(1511, 96)
(1511, 100)
(1511, 104)
(1511, 106)
(1511, 113)
(1511, 115)
(1511, 124)
(1511, 131)
(1511, 134)
(1511, 138)
(1511, 140)
(1511, 144)
(1511, 148)
(1511, 152)
(1511, 155)
(1511, 157)
(1511, 161)
(1511, 163)
(1511, 168)
(1511, 170)
(1511, 173)
(1511, 178)
(1511, 181)
(1511, 189)
(1511, 194)
(1511, 196)
(1511, 198)
(1511, 200)
(1511, 205)
(1511, 210)
(1511, 212)
(1511, 214)
(1511, 216)
(1511, 218)
(1511, 220)
(1511, 230)
(1511, 232)
(1511, 235)
(1511, 238)
(1511, 246)
(1511, 248)
(1511, 255)
(1511, 261)
(1511, 265)
(1511, 269)
(1511, 271)
(1511, 275)
(1511, 322)
(1511, 369)
(1511, 416)
(1511, 463)
(1511, 510)
(1511, 557)
(1511, 602)
(1511, 647)
(1511, 692)
(1511, 737)
(1511, 782)
(1511, 827)


In [72]:
dfxoh.drop(ohnames, axis=1, inplace=True)
print(dfxoh.shape)

(1511, 762)


In [73]:
drop_names = []
names = dfx_float.columns
for n in names:
    x = np.sum(dfx_float[n].isnull())
    if x > 500:
        drop_names.append(n)
print(len(drop_names))

0


In [74]:
means = []
stds = []
drop_names = []
names = dfx_float.columns
for n in names:
    c = dfx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    means.append(tmpm)
    stds.append(tmps)
    if tmps == 0:
        drop_names.append(n)
print(len(set(means)), len(set(stds)))
print(len(drop_names))

1679 1622
270


In [75]:
dfx_float.drop(drop_names, axis=1, inplace=True)
dfx_float.shape

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(1511, 3329)

In [76]:
# not move sames first, fill nan
names = dfx_float.columns
added = set()
sames = []
for i in range(len(names)):
    tmpsl = []
    tmpsl.append(i)
    for j in range(i+1, len(names)):
        if means[i] == means[j] and stds[i] == stds[j]:
            if i not in added:
                added.add(i)
            if j not in added:
                added.add(j)
                tmpsl.append(j)
    if len(tmpsl) > 1:
        sames.append(tmpsl)
print(len(sames))

363


In [77]:
names = dfx_float.columns
drop_names = []
for i in sames:
    for j in i[1:]:
        drop_names.append(names[j])
print(len(drop_names))

1677


In [78]:
print(dfx_float.shape)
trainx_float = copy.copy(dfx_float)
dfx_float.drop(drop_names, axis=1, inplace=True)
print(dfx_float.shape)

(1511, 3329)
(1511, 1652)


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [79]:
# fix first then log
names = dfx_float.columns
for n in names:
    c = dfx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    assert tmps>0
    dfx_float[n][c.isnull()] = tmpm
print(np.sum(dfx_float.isnull().values))

/home/magnusterra/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/magnusterra/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


0


In [80]:
mid = dfx_float-np.min(dfx_float)+1
dfx_log1p = np.log1p(mid)
del mid
print(np.sum(dfx_log1p.isnull().values))

0


In [81]:
for n in dfx_log1p.columns:
    c = dfx_log1p[n]
    tmpm = np.mean(c)
    tmps = np.std(c)
    assert tmps > 0
    dfx_log1p[n] = (dfx_log1p[n] - tmpm)/tmps

In [82]:
dfx = pd.concat([dfx_log1p, dfxoh, dfx_days], axis=1)
dfx.shape

(1511, 2500)

In [83]:
def MSE(y_raw, y_pred):
    return np.mean(np.square(y_raw-y_pred))

In [84]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV 

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [89]:
xgb = XGBRegressor(gamma=0.0001, max_depth=4, objective='reg:linear', subsample=0.85)
xgb.fit(dfx[:799].values, train_y.values)
test1_pred = xgb.predict(dfx[:799].values)
print(MSE(test1_pred, train_y.values))
test1_pred = xgb.predict(dfx[800:1100].values)
test1_pred = test1_pred.reshape(300)
save = pd.DataFrame({'ID':test1_id, 'value':test1_pred})
save.to_csv('answer/Synchronous_timeappend_logohtime_xgb_20180126.csv', index=False, header=False)

0.0012093681205
